In [4]:
#importing the libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# For Imputation
from sklearn.preprocessing import LabelEncoder

# For data preprocessing
from sklearn.model_selection import train_test_split, KFold, cross_validate

# For model building
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# For visualizing the descision tree
from sklearn import tree

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
#import the necessary modelling algos.
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score,auc  
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import (accuracy_score, f1_score, log_loss, roc_auc_score, roc_curve, confusion_matrix) 
from sklearn.model_selection import (cross_val_score, GridSearchCV, RandomizedSearchCV, learning_curve, validation_curve, train_test_split)  

#For performing a series of operations 
from sklearn.pipeline import make_pipeline 

from sklearn.metrics import plot_confusion_matrix 



#opening file 
df = pd.read_csv("oasis_longitudinal.csv")
df.head()


'''
Visit: Посещение, визит (например, в клинику или к врачу).
MR Delay: Задержка в магнитно-резонансной томографии.
Age: Возраст участника исследования.
EDUC: Уровень образования (например, количество лет обучения).
SES: Социально-экономический статус.
MMSE: Шкала когнитивных функций (Мини-метод оценки психического состояния).
CDR: Клиническая деменция Рейтинг (шкала оценки деменции).
eTIV: Оценочный объем внутренней черепной полости.
nWBV: Нормализованный объем мозга.
ASF: Масштабный фактор атласа (используется в нейровизуализации).
'''

df.info()
df.describe()
df.shape
df.isnull().sum()

#randomly assign left hand more than 20% of poeple 
df['Handedness'] = 'Right'
np.random.seed(42)  # Для воспроизводимости результатов
random_indices = np.random.choice(df.index, size=int(len(df) * 0.2), replace=False)
df.loc[random_indices, 'Handedness'] = 'Left'
df.loc[random_indices, 'Hand'] = 'L'

df.head(10)


#checking for the null and nan value 
df.isnull().sum()

numeric_df = df.select_dtypes(include=[np.number])
skewness = numeric_df.skew()

#replacing the null value with data from the median 
df.MMSE.fillna(df.MMSE.median(),inplace=True)
df.SES.fillna(df.SES.median(), inplace = True)

df.isnull().sum()

#cheking again the skew
numeric_df = df.select_dtypes(include=[np.number])
skewness = numeric_df.skew()
print(skewness)

'''
replacing the values for the SES value 
the values of CDR are the same for the evaluation of dimentia , that is why 
need to reverse the SES so that, SES = 1 (Lowest Status) and SES = 5 (Highest Status) of dimentia 

'''

ses_map = {5:1,4:2,3:3,2:4,1:5}
df.SES = df.SES.map(ses_map)
df.head()

df.SES.value_counts()

#saving the data in csv doc
df_copy = df.copy()
df.to_csv('oasis_longitude.csv')

#checking the data type 
df.dtypes

#encoding gender 
gender_map = {'M':0, 'F':1}
df['Gender'] = df['M/F'].map(gender_map)

df.head(15)


df.dtypes

#now when we have new data from the encoding (from m/f to gender and from handness to hand), we can drop initial m/f and handness
df.drop(columns='M/F',axis=1,inplace=True)
df.drop(columns='Handedness',axis=1,inplace=True)

df.head(12)

#cheking the amount of demented people, non-demented and converted 
df.Group.value_counts()


df['Group'].unique()
target_map = {'Nondemented': 0, 'Demented': 1, 'Converted': 2}
df['Group'] = df['Group'].map(target_map)

df.head(30)

#checing the correclation 
#EDA 

df.info()
df["MRI ID"].unique()


''''
from sklearn.preprocessing import LabelEncoder

# Создание экземпляра LabelEncoder
label_encoder = LabelEncoder()

# Применение LabelEncoder к столбцу 'MRI ID'
df['MRI ID Encoded'] = label_encoder.fit_transform(df['MRI ID'])

# Просмотр изменений
print(df[['MRI ID', 'MRI ID Encoded']].head())

corr = df.corr()
corr
'''

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Subject ID  373 non-null    object 
 1   MRI ID      373 non-null    object 
 2   Group       373 non-null    object 
 3   Visit       373 non-null    int64  
 4   MR Delay    373 non-null    int64  
 5   M/F         373 non-null    object 
 6   Hand        373 non-null    object 
 7   Age         373 non-null    int64  
 8   EDUC        373 non-null    int64  
 9   SES         354 non-null    float64
 10  MMSE        371 non-null    float64
 11  CDR         373 non-null    float64
 12  eTIV        373 non-null    int64  
 13  nWBV        373 non-null    float64
 14  ASF         373 non-null    float64
dtypes: float64(5), int64(5), object(5)
memory usage: 43.8+ KB
Visit       1.041023
MR Delay    0.945037
Age         0.141700
EDUC       -0.025977
SES         0.284587
MMSE       -2.375332
CDR      

"'\nfrom sklearn.preprocessing import LabelEncoder\n\n# Создание экземпляра LabelEncoder\nlabel_encoder = LabelEncoder()\n\n# Применение LabelEncoder к столбцу 'MRI ID'\ndf['MRI ID Encoded'] = label_encoder.fit_transform(df['MRI ID'])\n\n# Просмотр изменений\nprint(df[['MRI ID', 'MRI ID Encoded']].head())\n\ncorr = df.corr()\ncorr\n"